<a href="https://colab.research.google.com/github/libra3910/diveintocode-ml/blob/master/seq2seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 2.機械翻訳

```py
系列データに関する手法の基本的な活用例としては機械翻訳があります。これは系列データを入力し、系列データを出力する 
Sequence to Sequence の手法によって行えます。
```

### 【問題1】機械翻訳の実行とコードリーディング

```py
以下のサンプルコードは、短い英語からフランス語への変換を行うものです。これを動かしてください。
keras/lstm_seq2seq.py at master · rstudio/keras

その上でこのサンプルコードの各部分がどういった役割かを読み取り、まとめてください。以下のようにどこからどこの行が何をしているかを記述してください。
（例）
51から55行目 : ライブラリのimport
57から62行目 : ハイパーパラメータの設定

《文字単位のトークン化》
この実装ではテキストのベクトル化の際に、単語ではなく文字ごとを1つのトークンとして扱っています。

scikit-learnでBoWを計算するCountVectorizerの場合では、デフォルトの引数はanalyzer=’word’で単語を扱いますが、charやchar_wbとすることで文字を扱えるようになります。
charとchar_wbの2種類の方法があり、char_wbを指定した場合、n_gramが単語内からのみ作成されます。逆にcharは単語の区切りが関係なくn_gramが作成されます。This movie is very good.というテキストを3-gramでカウントする時、charではs mやe iといった単語をまたぐ数え方もしますが、char_wbではこれらを見ません。
sklearn.feature_extraction.text.CountVectorizer — scikit-learn 0.21.3 documentation
```

```py
Sequence to sequence example in Keras (character-level).
This script demonstrates how to implement a basic character-level
sequence-to-sequence model. We apply it to translating
short English sentences into short French sentences,
character-by-character. Note that it is fairly unusual to
do character-level machine translation, as word-level
models are more common in this domain.
# Summary of the algorithm:
- We start with input sequences from a domain (e.g. English sentences)
    and correspding target sequences from another domain
    (e.g. French sentences).
- An encoder LSTM turns input sequences to 2 state vectors
    (we keep the last LSTM state and discard the outputs).
- A decoder LSTM is trained to turn the target sequences into
    the same sequence but offset by one timestep in the future,
    a training process called "teacher forcing" in this context.
    Is uses as initial state the state vectors from the encoder.
    Effectively, the decoder learns to generate `targets[t+1...]`
    given `targets[...t]`, conditioned on the input sequence.
- In inference mode, when we want to decode unknown input sequences, we:
    - Encode the input sequence into state vectors
    - Start with a target sequence of size 1
        (just the start-of-sequence character)
    - Feed the state vectors and 1-char target sequence
        to the decoder to produce predictions for the next character
    - Sample the next character using these predictions
        (we simply use argmax).
    - Append the sampled character to the target sequence
    - Repeat until we generate the end-of-sequence character or we
        hit the character limit.
# Data download:
English to French sentence pairs.
https://www.manythings.org/anki/fra-eng.zip
Lots of neat sentence pairs datasets can be found at:
https://www.manythings.org/anki/
# References:
- Sequence to Sequence Learning with Neural Networks
    https://arxiv.org/abs/1409.3215
- Learning Phrase Representations using
    RNN Encoder-Decoder for Statistical Machine Translation
    https://arxiv.org/abs/1406.1078
```

### 1.ライブラリのimport　print関数、kerasモデルとレイア、numpy

In [ ]:
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

### 2.ハイパーパラメータの設定

In [ ]:
batch_size = 64  # Batch size for training.
epochs = 3  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'fra-eng/fra.txt'

### 3.テキストデータの読み込み、英語、フランス語ともトークンのリストを作成する

In [ ]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
lines = open(data_path, encoding='utf-8').read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 69
Number of unique output tokens: 93
Max sequence length for inputs: 16
Max sequence length for outputs: 59


### 4.inputおよびtargetのから行列を作成する。それぞれセンテンス分、トークン数の最大値をトークン数分、確保する。

In [ ]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

### 5.サンプル行数（10000行）分、テキストのトークンの文字と一致するデータに1を立てる。encorder：input、decorder：input、target

In [ ]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

### 6.encoderとdecorderの定義

In [ ]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

### 7.modelの定義：encorder、decorderの入力、decorderの出力。コンパイルし、学習。特徴量：encorder（英語）、ターゲット：decorder（フランス語）

In [ ]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('s2s.h5')

Epoch 1/3
125/125 [==============================] - 49s 343ms/step - loss: 1.0944 - val_loss: 1.1319
Epoch 2/3
125/125 [==============================] - 38s 302ms/step - loss: 0.9695 - val_loss: 1.0965
Epoch 3/3
125/125 [==============================] - 39s 313ms/step - loss: 0.9188 - val_loss: 1.0466


### 8.encorderモデルとdecorderモデルの定義

In [ ]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

### 9.文字を数字で検索できるようにする。

In [ ]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

### 10.英語のセンテンスを入力し、フランス語のセンテンスを返す。

①　encorder_modelで予測した結果を、decorderの予測時の入力とする。\
②　decorderの予測結果をもとに、上記のreverse_target_char_indexから文字を検索し、\
③　decoded_sentenceに加算する。\
④　decoded_sentence（フランス語の翻訳結果）を返す。

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

### 11.100センテンス分の翻訳を実施する。

In [ ]:
for seq_index in range(100):
    # Take one sequence (part of the training test)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: Go.
Decoded sentence: ente                                                        
-
Input sentence: Hi.
Decoded sentence: Aren e                                                      
-
Input sentence: Run!
Decoded sentence: e te                                                        
-
Input sentence: Run!
Decoded sentence: e te                                                        
-
Input sentence: Wow!
Decoded sentence: Tome sous                                                   
-
Input sentence: Fire!
Decoded sentence: Arren                                                       
-
Input sentence: Help!
Decoded sentence: Arren                                                       
-
Input sentence: Jump.
Decoded sentence: à pair                                                      
-
Input sentence: Stop!
Decoded sentence: Arente                                                      
-
Input sentence: Stop!
Decoded sentence: Arente                                

-
Input sentence: Be fair.
Decoded sentence: Tome sous                                                   
-
Input sentence: Be fair.
Decoded sentence: Tome sous                                                   
-
Input sentence: Be fair.
Decoded sentence: Tome sous                                                   
-
Input sentence: Be kind.
Decoded sentence: Tome sous                                                   
-
Input sentence: Be nice.
Decoded sentence: Tome suis                                                   
-
Input sentence: Be nice.
Decoded sentence: Tome suis                                                   
-
Input sentence: Be nice.
Decoded sentence: Tome suis                                                   
-
Input sentence: Be nice.
Decoded sentence: Tome suis                                                   
-
Input sentence: Be nice.
Decoded sentence: Tome suis                                                   
-
Input sentence: Be nice.
Decoded sentence: T

### 【問題2】イメージキャプショニングの学習済みモデルの実行

### README.mdに従い、「1. Clone the repositories」を実行

In [1]:
!git clone https://github.com/pdollar/coco.git

Cloning into 'coco'...
remote: Enumerating objects: 975, done.
remote: Total 975 (delta 0), reused 0 (delta 0), pack-reused 975
Receiving objects: 100% (975/975), 11.72 MiB | 18.58 MiB/s, done.
Resolving deltas: 100% (576/576), done.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# カレントディレクトリの変更

import os

os.chdir('/content/drive/My Drive/')

# カレントディレクトリの取得
print(os.getcwd())

/content/drive/My Drive


In [4]:
cd coco/PythonAPI/

/content/drive/My Drive/coco/PythonAPI


In [5]:
pwd

'/content/drive/My Drive/coco/PythonAPI'

In [7]:
!make

python setup.py build_ext --inplace
running build_ext
cythoning pycocotools/_mask.pyx to pycocotools/_mask.c
/usr/local/lib/python3.7/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/drive/MyDrive/coco/PythonAPI/pycocotools/_mask.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
building 'pycocotools._mask' extension
creating build
creating build/common
creating build/temp.linux-x86_64-3.7
creating build/temp.linux-x86_64-3.7/pycocotools
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fdebug-prefix-map=/build/python3.7-a56wZI/python3.7-3.7.10=. -fstack-protector-strong -Wformat -Werror=format-security -g -fdebug-prefix-map=/build/python3.7-a56wZI/python3.7-3.7.10=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/local/lib/python3.7/dist-packages/numpy/

In [9]:
!python setup.py build

running build
running build_py
creating build
creating build/lib.linux-x86_64-3.7
creating build/lib.linux-x86_64-3.7/pycocotools
copying pycocotools/cocoeval.py -> build/lib.linux-x86_64-3.7/pycocotools
copying pycocotools/mask.py -> build/lib.linux-x86_64-3.7/pycocotools
copying pycocotools/coco.py -> build/lib.linux-x86_64-3.7/pycocotools
copying pycocotools/__init__.py -> build/lib.linux-x86_64-3.7/pycocotools
running build_ext
skipping 'pycocotools/_mask.c' Cython extension (up-to-date)
building 'pycocotools._mask' extension
creating build/common
creating build/temp.linux-x86_64-3.7
creating build/temp.linux-x86_64-3.7/pycocotools
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fdebug-prefix-map=/build/python3.7-a56wZI/python3.7-3.7.10=. -fstack-protector-strong -Wformat -Werror=format-security -g -fdebug-prefix-map=/build/python3.7-a56wZI/python3.7-3.7.10=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_F

In [10]:
!python setup.py install

running install
running bdist_egg
running egg_info
creating pycocotools.egg-info
writing pycocotools.egg-info/PKG-INFO
writing dependency_links to pycocotools.egg-info/dependency_links.txt
writing requirements to pycocotools.egg-info/requires.txt
writing top-level names to pycocotools.egg-info/top_level.txt
writing manifest file 'pycocotools.egg-info/SOURCES.txt'
writing manifest file 'pycocotools.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
running build_ext
skipping 'pycocotools/_mask.c' Cython extension (up-to-date)
creating build/bdist.linux-x86_64
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/pycocotools
copying build/lib.linux-x86_64-3.7/pycocotools/cocoeval.py -> build/bdist.linux-x86_64/egg/pycocotools
copying build/lib.linux-x86_64-3.7/pycocotools/mask.py -> build/bdist.linux-x86_64/egg/pycocotools
copying build/lib.linux-x86_64-3.7/pycocotools/coco.py -> build/bdist.linux-x86_6

In [11]:
cd ../../

/content/drive/My Drive


In [13]:
!git clone https://github.com/yunjey/pytorch-tutorial.git

Cloning into 'pytorch-tutorial'...
remote: Enumerating objects: 917, done.
remote: Total 917 (delta 0), reused 0 (delta 0), pack-reused 917
Receiving objects: 100% (917/917), 12.80 MiB | 13.35 MiB/s, done.
Resolving deltas: 100% (490/490), done.


In [30]:
cd pytorch-tutorial/tutorials/03-advanced/image_captioning/

[Errno 2] No such file or directory: 'pytorch-tutorial/tutorials/03-advanced/image_captioning/'
/content/drive/My Drive/pytorch-tutorial/tutorials/03-advanced/image_captioning


In [15]:
!pip install -r requirements.txt

In [27]:
pwd

'/content/drive/My Drive/pytorch-tutorial/tutorials/03-advanced/image_captioning'

### 重みおよびボキャブラリファイルをダウンロードし、それぞれ./modelsおよび./dataに保存する。

### 「5. Test the model」を行う。python sample.py --image='png/example.png'を実行。

In [29]:
pwd

'/content/drive/My Drive/pytorch-tutorial/tutorials/03-advanced/image_captioning'

In [32]:
!python sample.py --image='png/example.png'

Downloading: "https://download.pytorch.org/models/resnet152-b121ed2d.pth" to /root/.cache/torch/hub/checkpoints/resnet152-b121ed2d.pth
100% 230M/230M [00:02<00:00, 82.6MB/s]
<start> a group of giraffes standing next to each other . <end>


In [35]:
!python sample.py --image='png/cat1.jpg'

<start> a cat is sitting on a couch next to a cat . <end>


In [36]:
!python sample.py --image='png/dog5.jpg'

<start> a small dog laying on a couch next to a remote . <end>


In [38]:
!python sample.py --image='png/DSC_0122.JPG'

<start> a group of people standing on a beach with a kite . <end>


### 【問題3】Kerasで動かしたい場合はどうするかを調査

```py
Pytorch → kerasへの変換方法
1.MMdnn
　Microsoft Researchにより開発が進められているオープンソースの深層学習モデルの変換と可視化を行うツール。
中間表現を経由することで様々なフレームワーク間でのモデルデータの相互変換を実現。

サポートしているフレームワーク。
Caffe、Keras、Tensorflow、CNTK、MXNet、PyTorch、CoreML

以下のコマンドを実行すれば、ソースコーディングすることなく、
pytorchからonnx経由でkerasに変換することが可能
$ mmconvert -sf pytorch -iw imagenet_inception_v3.h5 -df onnx -om imagenet_inception_v3.keras
$ mmconvert -sf onnx -iw imagenet_inception_v3.h5 -df keras -om imagenet_inception_v3.keras


-sf [framework]・・・・変換元(sauce)のフレームワーク
-iw [ファイル]・・・・・入力する重みファイル
-in [ファイル]・・・・・入力するネットワークファイル
-df [framework]・・・・変換先(destination)フレームワーク
-om [ファイル]・・・・・出力先のモデルファイル名
2.mobilenetv3_large()を使って、pytorch→ONNI→kerasに変換する

```